# Light curve

In this notebook we'll assemble the light curve of a star, calibrated against other stars of the same field, with the purpose of uncovering an exoplanet transit.

As usual, you can download this page as a {download}`jupyter notebook <./lightcurve.ipynb>` file.

In [ ]:
import dataredconfig
import pathlib

import numpy as np
import pandas as pd
import astropy
import astropy.table
import astropy.visualization
from astropy import units as u

import datetime

%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt

import ccdproc

In [ ]:
# We'll ignore some astropy warnings that get raised as our FITS headers (from NINA) are not 100% standards compliant.
import warnings
warnings.simplefilter('ignore', category=astropy.wcs.FITSFixedWarning)

In [ ]:
photometry_dir = dataredconfig.work_dir / "PHOTOMETRY"

object_to_process = "HD92670"

catalog_filepaths = sorted(list(photometry_dir.glob('*.fits')))
catalogs = []

# We'll read all catalogs in, can keep only those matching the desired object in the above list.
for catalog_filepath in catalog_filepaths:
    
    catalog = astropy.table.Table.read(catalog_filepath)

    # We select the photometric catalogs of our object:  
    if "OBJECT" in catalog.meta:
        if catalog.meta["OBJECT"] == object_to_process:
            print(f"{catalog_filepath} : {catalog.meta}")
            catalogs.append(catalog)


In [ ]:
# We combine these catalogs into a single table, in "depth": columns will be 2D, where the second dimension is time.
catalog = astropy.table.dstack(catalogs, join_type="exact", metadata_conflicts="silent")

# We also produce a list of datetime objects, from the FITS headers:
date_strings = [c.meta["DATE-OBS"] for c in catalogs]
dates = [datetime.datetime.fromisoformat(date) for date in date_strings]

# And while we are at it, same for the airmass:
airmasses = [c.meta["AIRMASS"] for c in catalogs]

# We read the reference catalog, as this one contains the position of each star
ref_catalog = astropy.table.Table.read(photometry_dir / f"ref_catalog_{object_to_process}.fits")
assert len(ref_catalog) == len(catalog) # Just a check that these are indeed of same length

# We copy the positions from the reference catalog over to our combined catalog:
catalog["sky_centroid_win"] = ref_catalog["sky_centroid_win"]

n_epochs = len(dates)
print(f"Number of epochs: {n_epochs}")

# And just for information, we print the column names and column shapes of our "combined" catalogue.
# Indeed, most of the "columns" are in fact 2D arrays: 
for colname in catalog.colnames:
    print(f"{colname}: {catalog[colname].shape}")



In [ ]:
# Optional cell to create a binned version of the data catalog

binsize = 10 # How many measurements to group in each bin

n_epochs_binnable = n_epochs - (n_epochs % binsize) # This is number of epochs we can use (largest possible integer multiple of binsize)
print(f"With bin size {binsize} we can use {n_epochs_binnable} of the {n_epochs} epochs.")

# First, we create a list of binned dates: we take the "mean" of the epochs in each bin.
dates_reshape = np.reshape(dates[:n_epochs_binnable], (-1, binsize))
dates_binned = [pd.to_datetime(pd.Series(bin)).mean() for bin in dates_reshape]

# Then we create a "binned" catalog, starting from a full copy.
catalog_binned = catalog.copy()

# Define how to process columns:
colnames_to_sum = ["sum_4", "sum_6", "sum_8", "sum_10", "back_sum_4", "back_sum_6", "back_sum_8", "back_sum_10"] # Could add "flux_fit" if available
colnames_to_mean = [] # Could add "fwhm_fit", "q_fit"
colnames_to_median = ["max_4", "max_6", "max_8", "max_10"]

for colname in colnames_to_sum + colnames_to_mean + colnames_to_median:

    original_column_shape = catalog[colname].shape
    if len(original_column_shape) != 2:
        raise(RuntimeError(f"Column {colname} has shape {original_column_shape} and can't be binned."))
    
    nb_sources = original_column_shape[0]
    column_reshape = np.reshape(catalog[colname][:,:n_epochs_binnable], (nb_sources, -1, binsize)) # This is now 3D : (source index, nb of binned epochs, binsize)

    if colname in colnames_to_sum:
        column_binned = np.sum(column_reshape, axis=2) #sum within bins
    elif colname in colnames_to_mean:
        column_binned = np.mean(column_reshape, axis=2) # mean within bins
    elif colname in colnames_to_median:
        column_binned = np.median(column_reshape, axis=2) # median within bins

    catalog_binned[colname] = column_binned

# For information, the column names and column shapes of the "binned" catalogue are:
for colname in catalog_binned.colnames:
    print(f"{colname}: {catalog_binned[colname].shape}")
print(f"Number of epochs in binned catalog: {len(dates_binned)}")


We display an image of the field, overplotting the "source indices" corresponding to rows of our catalog

In [ ]:
# We load one of the images, it does not have to be a specific one.
light_prered_dir = dataredconfig.work_dir / "ASTROMETRY"
science_files = ccdproc.ImageFileCollection(light_prered_dir, keywords=dataredconfig.ifc_header_keywords)
science_files = science_files.filter(object=object_to_process)
image_path = science_files.files[0]
image = ccdproc.CCDData.read(image_path, unit="adu")
image.data -= np.median(image.data) # Quick sky subtraction

# We test that the selected image does have WCS information, to prevent unexpected errors being raised further below. 
if(image.wcs is None):
    raise RuntimeError("This image has no WCS, make sure you specify the correct directory (i.e., with WCS) above!")


In [ ]:

# And now create the figure
plt.figure(figsize=(10, 6))
ax = plt.subplot(projection=image.wcs)
ax.imshow(image.data, origin='lower', cmap='Greys_r', interpolation='nearest',
    norm=astropy.visualization.simple_norm(image.data, stretch="sqrt", vmin=-20, vmax=500))
ax.scatter(
    catalog["sky_centroid_win"].ra.degree,
    catalog["sky_centroid_win"].dec.degree,
    transform=ax.get_transform('world'),
    s=50, # The size of these markers is not related to any measurement apertures!
    edgecolor='red', facecolor='none'
    )
for line in catalog:
    ax.text(
        x=line["sky_centroid_win"].ra.degree,
        y=line["sky_centroid_win"].dec.degree,
        s=str(line.index),
        transform=ax.get_transform('world'),
        color="cyan"
        )
ax.grid(color='white', ls='solid')
ax.coords[0].set_axislabel('RA')
ax.coords[1].set_axislabel('Dec')
#ax.coords[0].set_ticks(spacing=5.*u.arcmin)
#ax.coords[1].set_ticks(spacing=5.*u.arcmin)
plt.tight_layout()
plt.show()



Let's now visualize the raw light curves, to get a first impression.

In [ ]:
# Which stars ("source indices") should we plot?
indices_show = []

# We compute an "instrumental magnitude" from our flux measurements:
catalog["instr_mag"] = -2.5 * np.log10(catalog["sum_10"].value) # this is a "2D" column: (source index, date)

plt.figure()
ax = plt.subplot()

# Plot some field stars in grey:
for index in indices_show:
    ax.plot(dates, catalog["instr_mag"][index], lw=1, label=index)

ax.invert_yaxis() # Needed, as we show a magnitude on y.

# Some advanced settings to help getting a nice format of the date axis labels:
ax.xaxis.set_major_formatter(matplotlib.dates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

ax.set_ylabel("Instrumental magnitude")
plt.gcf().autofmt_xdate()
plt.legend()

plt.show()

```{admonition} Question
Do you observe any trends in these light curves, and are these common to all stars? Comment on what could cause these trends.
```

## Calibration

To reveal the transit itself, some empirical calibration of the flux (or magnitude) measured in each exposure is needed.

```{admonition} Question
What would be the properties of good reference stars to use for the calibration?
```

In [ ]:
# For the calibration (and to produce a nice figure afterwards) some "settings":

target_index = 611 # Which index in the field image from above corresponds to the target star (the one that shows the transit)?
mean_instr_mag_limit = -15.0
std_instr_mag_limit = 0.015
nbr_images_baseline = 20 # how many images first baseline (to set magnitude)
n_bright_stars_calib = 20 # How many of the brightest stars to use for calibration
n_bright_stars_show = 8 # How many of the brightest stars should be plotted 
title_str = "HAT-P-23 b, Moon 34% @ 119°, depth 10.5 ppt"

flux_colname = "sum_6"



# Some values from the literature:
target_mag = 11.9 # What magnitude does this star have?
transit_depth_ppt = 10.5 # How deep is the transit (in parts per thousands "ppt") 
ingress_datetime = datetime.datetime.fromisoformat("2024-08-27T20:31") # Both in UTC
egress_datetime = datetime.datetime.fromisoformat("2024-08-27T22:42")


transit_depth_mag = 2.5*np.log10(1.0/(1.0-transit_depth_ppt/1000.0))



# We compute the instrumental magnitude, for the desired aperture size (Maybe you did this already, but it doesn't harm)
catalog["instr_mag"] = -2.5 * np.log10(catalog[flux_colname].value) # a "2D" column (index, date)
catalog["median_instr_mag"] = np.nanmedian(catalog["instr_mag"], axis=1) # one value per source
catalog["std_instr_mag"] = np.std(catalog["instr_mag"].value, axis=1) # this is just a 1Dcolumn: (source index)

target_center_pos = catalog["sky_centroid_win"][target_index]
catalog["separation"] = target_center_pos.separation(catalog["sky_centroid_win"])



catalog["flux_sort_indices"] = np.argsort(catalog["median_instr_mag"])

flux_sort_indices = [i for i in catalog["flux_sort_indices"] if 
                     catalog["median_instr_mag"][i] > mean_instr_mag_limit and 
                     catalog["std_instr_mag"][i] < std_instr_mag_limit and
                     catalog["separation"][i] < 0.1 and
                     i != target_index
                     #np.abs(catalog["med_diff"][i] - -0.01764350479282406) < 0.005
                    ]

print("Number of stars remaining:", len(flux_sort_indices))
indices_calib = flux_sort_indices[0:n_bright_stars_calib]
indices_show = flux_sort_indices[0:n_bright_stars_show]

#catalog["flux_sort_indices"] = np.argsort(catalog["median_instr_mag"])
#indices_calib = catalog["flux_sort_indices"][0:n_bright_stars_calib]
#indices_show = catalog["flux_sort_indices"][0:n_bright_stars_show]

#indices_calib = [118, 112, 103, 136, 141]
#indices_show = [118, 112, 103, 136, 141]

catalog["delta_instr_mag"] = catalog["instr_mag"] - np.expand_dims(catalog["median_instr_mag"], axis=1) # 2D - 1D -> 2D (index, date)

same_calib_for_all = np.nanmedian(catalog["delta_instr_mag"][indices_calib], axis=0)
# Normalize this:
same_calib_for_all -= np.mean(same_calib_for_all)
# And make it a colum of the catalog
catalog["rel_calib"] = np.tile(same_calib_for_all, (len(catalog), 1))

catalog["calib_instr_mag"] = catalog["instr_mag"] - catalog["rel_calib"]


zero_point = target_mag - np.nanmedian(catalog["calib_instr_mag"].value[target_index, 0:nbr_images_baseline])
print(np.nanmedian(catalog["calib_instr_mag"].value[target_index, 0:nbr_images_baseline]))
print(f"Zero-point: {zero_point}")
catalog["mag"] = catalog["calib_instr_mag"] + zero_point







### Same for the binned catalog


nbr_images_baseline = 4

catalog_binned["instr_mag"] = -2.5 * np.log10(catalog_binned[flux_colname].value) # a "2D" column (index, date)
catalog_binned["median_instr_mag"] = np.nanmedian(catalog_binned["instr_mag"], axis=1) # one value per source

#catalog_binned["flux_sort_indices"] = np.argsort(catalog_binned["median_instr_mag"])
#indices_calib = catalog_binned["flux_sort_indices"][0:n_bright_stars_calib]
#indices_show = catalog_binned["flux_sort_indices"][0:n_bright_stars_show]

#indices_calib = [118, 112, 103, 136, 141]
#indices_show = [118, 112, 103, 136, 141]

catalog_binned["delta_instr_mag"] = catalog_binned["instr_mag"] - np.expand_dims(catalog_binned["median_instr_mag"], axis=1) # 2D - 1D -> 2D (index, date)

same_calib_for_all = np.nanmedian(catalog_binned["delta_instr_mag"][indices_calib], axis=0)
# Normalize this:
same_calib_for_all -= np.mean(same_calib_for_all)
# And make it a colum of the catalog
catalog_binned["rel_calib"] = np.tile(same_calib_for_all, (len(catalog_binned), 1))

catalog_binned["calib_instr_mag"] = catalog_binned["instr_mag"] - catalog_binned["rel_calib"]


zero_point = target_mag - np.nanmedian(catalog_binned["calib_instr_mag"].value[target_index, 0:nbr_images_baseline])
print(np.nanmedian(catalog_binned["calib_instr_mag"].value[target_index, 0:nbr_images_baseline]))
print(f"Zero-point: {zero_point}")
catalog_binned["mag"] = catalog_binned["calib_instr_mag"] + zero_point




In [ ]:
                    
plt.figure()#figsize=(6, 6))
ax = plt.subplot()
ax.axvline(ingress_datetime, lw=1)
ax.axvline(egress_datetime, lw=1)
ax.axhline(target_mag, lw=1, ls="--")
ax.axhline(target_mag + transit_depth_mag, lw=1, ls=":")

# Plot some field stars in grey:
for (i, index) in enumerate(indices_show):
    if index == target_index:
        continue
    ax.plot(
    dates, catalog["mag"][index] - (catalog["median_instr_mag"].value[index] - catalog["median_instr_mag"].value[target_index]) - 0.012*(i+3), 
    #dates, catalog["mag"][index] - catalog["calib"][target_index], 
    lw=0.3
    )

# Plot the target in red:
ax.plot(
    dates, catalog["mag"][target_index], 
    color="red", ls="None", marker="o", markersize=1, alpha=0.2, lw=0.1
    )

# compute error bars

gain =  0.376 # e/ADU
flux = catalog_binned[flux_colname][target_index]
#flux = catalog[flux_colname][target_index]
flux_err = np.sqrt(flux) / (np.sqrt(gain))
mag_err = 2.5 * np.log10(1 + (flux_err/flux))


ax.errorbar(dates_binned, catalog_binned["mag"][target_index], yerr=mag_err, ls="None", marker="o", color="red", markersize=2)
#ax.errorbar(dates, catalog["mag"][target_index], yerr=mag_err, ls="None", marker="o", color="red", markersize=2)



ax.invert_yaxis()

ax2 = ax.twinx()
color="purple"
ax2.set_ylabel('Airmass', color=color)
ax2.plot(dates, airmasses, color=color, lw=1, ls="-.")
ax2.tick_params(axis='y', labelcolor=color)


ax.xaxis.set_major_formatter(matplotlib.dates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
#plt.colorbar(label=f"Separation to target center in {cat['separation'].unit}")
#plt.xlabel("Date")
ax.set_ylabel("r Magnitude", color="red")
ax.set_xlabel("UTC")
plt.gcf().autofmt_xdate()
plt.title(title_str)
plt.tight_layout()
#plt.show()
plt.savefig("2024-08-27_HAT-P-23_b.pdf")
